In [1]:
%run imports_and_functions.ipynb

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Constructing All Cities Dataset

In [2]:
X_train2 = joblib.load('X_trainBoston.joblib')
y_train2 = joblib.load('y_trainBoston.joblib')

X_train3 = joblib.load('X_trainMiami.joblib')
y_train3 = joblib.load('y_trainMiami.joblib')

X_train4 = joblib.load('X_trainSeattle.joblib')
y_train4 = joblib.load('y_trainSeattle.joblib')

In [3]:
print(X_train2.shape, X_train3.shape, X_train4.shape)

(2400, 1482) (4355, 1482) (2747, 1482)


In [5]:
X_allcities = X_train2.append(X_train3).append(X_train4)
X_allcities.shape

(9502, 1482)

In [4]:
y_allcities = y_train2.append(y_train3).append(y_train4)
y_allcities.shape

(9502,)

In [6]:
X_allcities.index = y_allcities.index
X_allcities.head()

,landarea,waterarea,0_restaurants_near,0_restaurants_far,1_restaurants_near,1_restaurants_far,2_restaurants_near,2_restaurants_far,3_restaurants_near,3_restaurants_far,4_restaurants_near,4_restaurants_far,0_shopping_near,0_shopping_far,1_shopping_near,1_shopping_far,2_shopping_near,2_shopping_far,3_shopping_near,3_shopping_far,4_shopping_near,4_shopping_far,0_food_near,0_food_far,1_food_near,1_food_far,2_food_near,2_food_far,3_food_near,3_food_far,4_food_near,4_food_far,0_beautysvc_near,0_beautysvc_far,1_beautysvc_near,1_beautysvc_far,2_beautysvc_near,2_beautysvc_far,3_beautysvc_near,3_beautysvc_far,4_beautysvc_near,4_beautysvc_far,0_homeservices_near,0_homeservices_far,1_homeservices_near,...,2_hair_extensions_far,3_hair_extensions_near,3_hair_extensions_far,4_hair_extensions_near,4_hair_extensions_far,0_lawyers_near,0_lawyers_far,1_lawyers_near,1_lawyers_far,2_lawyers_near,2_lawyers_far,3_lawyers_near,3_lawyers_far,4_lawyers_near,4_lawyers_far,0_french_near,0_french_far,1_french_near,1_french_far,2_french_near,2_french_far,3_french_near,3_french_far,4_french_near,4_french_far,0_selfstorage_near,0_selfstorage_far,1_selfstorage_near,1_selfstorage_far,2_selfstorage_near,2_selfstorage_far,3_selfstorage_near,3_selfstorage_far,4_selfstorage_near,4_selfstorage_far,0_physicaltherapy_near,0_physicaltherapy_far,1_physicaltherapy_near,1_physicaltherapy_far,2_physicaltherapy_near,2_physicaltherapy_far,3_physicaltherapy_near,3_physicaltherapy_far,4_physicaltherapy_near,4_physicaltherapy_far
250250001002006,0,0,1,7,1,55,3,93,0,0,0,1,3,46,0,5,4,30,0,22,0,0,0,4,6,22,2,58,2,0,0,0,0,3,0,10,1,38,0,0,0,0,10,121,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0
250250001002007,0,0,1,12,1,76,3,113,0,0,0,1,3,46,0,5,6,43,0,25,0,0,0,10,4,50,3,64,1,1,0,0,0,18,0,12,1,50,0,0,0,0,10,128,0,...,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0
250250001003000,0,0,1,10,1,80,1,126,0,0,0,1,3,49,0,5,6,44,0,24,0,0,0,10,6,51,2,67,2,0,0,0,0,18,0,12,1,50,0,0,0,0,10,133,0,...,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0
250250001003001,0,0,1,14,1,83,3,129,0,0,1,0,0,56,0,8,6,48,0,25,0,0,0,10,6,53,2,75,2,0,0,0,0,22,0,12,0,57,0,0,0,0,11,139,0,...,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,10,0,0,0,0,0,0,0,0,1,5,0,0,0,0,0,0,0,0
250250001003002,0,0,1,30,1,123,3,146,0,0,1,4,0,61,0,8,19,49,4,21,0,0,0,14,5,62,2,95,1,1,0,0,0,22,1,21,12,54,0,0,0,0,15,150,0,...,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,10,0,0,0,0,0,0,0,0,2,4,0,0,0,0,0,0,0,0


In [8]:
rowstodrop = y_allcities[y_allcities.astype(str) == '-'].index

In [10]:
print(y_allcities.shape)
y_allcities.drop(rowstodrop, axis=0, inplace=True)
y_allcities.shape

(9502,)


(9502,)

In [11]:
nantodrop = y_allcities[y_allcities.isnull()].index

In [12]:
nantodrop = y_allcities[y_allcities.isna()].index
print(X_allcities.shape)
X_allcities.drop(nantodrop, axis=0, inplace=True)
X_allcities.shape

(9502, 1482)


(9502, 1482)

In [13]:
print(y_allcities.shape)
y_allcities.drop(nantodrop, axis=0, inplace=True)
y_allcities.shape

(9502,)


(9502,)

In [14]:
y_allcities = y_allcities.map(float)

y_allcities.head()

250250001002006    45835.0
250250001002007    45835.0
250250001003000    41421.0
250250001003001    41421.0
250250001003002    41421.0
Name: income, dtype: float64

In [15]:
joblib.dump(X_allcities, 'Xtrain_2nd_cities.joblib', compress=9)

['Xtrain_2nd_cities.joblib']

In [16]:
joblib.dump(y_allcities, 'ytrain_2nd_cities.joblib', compress=9)

['ytrain_2nd_cities.joblib']

## Testing Varied ML Models on All Cities Dataset

In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, RandomForestRegressor, BaggingClassifier
from sklearn.svm import SVR
from sklearn.metrics import f1_score, mean_squared_error

In [19]:
X_allcities = joblib.load('./2nd_cities/Xtrain_2nd_cities.joblib')
y_allcities = joblib.load('./2nd_cities/ytrain_2nd_cities.joblib')

In [57]:
X_features = joblib.load('X_features.joblib')

In [20]:
def accuracy_scorer(model, X_train, X_test, y_train, y_test):
    
    y_pred = pd.Series(data=model.predict(X_train),dtype=int)
    accuracy_train = f1_score(y_true = y_train,
                        y_pred = y_pred, average=None)
    
    y_pred = pd.Series(data=model.predict(X_test),dtype=int)
    accuracy_test = f1_score(y_true = y_test,
                       y_pred = y_pred, average=None)
    
    r2_scores = (model.score(X_train, y_train), model.score(X_test, y_test))
    
    print("R2 scores for " + str(type(model)) + " train set: " + str(r2_scores[0]))
    print("R2 scores for " + str(type(model)) + " test set: " + str(r2_scores[1]))
    print("The training accuracy-score for " + str(type(model)) + " is: ")
    print(str(accuracy_train))
    print("The testing accuracy-score for " + str(type(model)) + " is: ")
    print(str(accuracy_test))
    print()
    return (accuracy_train, accuracy_test)

### Preliminary Regressions (income still continuous)

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X_allcities, y_allcities, 
                                                    test_size=.15, random_state=42) 
X_train.shape

(9457, 1482)

In [62]:

ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

## Applying PCA to Xs
pca = PCA()
pca = pca.fit(X_train_scaled)
Z_train = pca.transform(X_train_scaled)
Z_test =pca.transform(X_test_scaled)

In [ ]:
linear_reg_PCA = LinearRegression()
linear_reg_PCA.fit(Z_train, y_train)

knn_reg_PCA = KNeighborsRegressor()
knn_reg_PCA.fit(Z_train, y_train)

cart_reg_PCA = DecisionTreeRegressor()
cart_reg_PCA.fit(Z_train, y_train)

In [63]:
bagged_reg_PCA = BaggingRegressor()
bagged_reg_PCA.fit(Z_train, y_train)

adaboost_reg_PCA = AdaBoostRegressor()
adaboost_reg_PCA.fit(Z_train, y_train)

random_forest_reg_PCA = RandomForestRegressor()
random_forest_reg_PCA.fit(Z_train, y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None)

In [66]:
support_vector_reg_PCA = SVR()
support_vector_reg_PCA.fit(Z_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [67]:
pca_reg_models = [linear_reg_PCA, knn_reg_PCA, cart_reg_PCA, bagged_reg_PCA, 
              adaboost_reg_PCA, random_forest_reg_PCA, support_vector_reg_PCA]

In [69]:
def rmse_scorer(model, X_train, X_test, y_train, y_test):
    mse_train = mean_squared_error(y_true = y_train,
                                  y_pred = model.predict(X_train))
    mse_test = mean_squared_error(y_true = y_test,
                                  y_pred = model.predict(X_test))
    rmse_train = mse_train ** 0.5
    rmse_test = mse_test ** 0.5

    r2_scores = (model.score(X_train, y_train), model.score(X_test, y_test))
    
    print("R2 scores for " + str(type(model)) + " train set: " + str(r2_scores[0]))
    print("R2 scores for " + str(type(model)) + " test set: " + str(r2_scores[1]))
    print("The training RMSE for " + str(type(model)) + " is: ")
    print(str(rmse_train))
    print("The testing RMSE for " + str(type(model)) + " is: ")
    print(str(rmse_test))
    print()
    return (rmse_train, rmse_test)

In [70]:
for model in pca_reg_models :
    rmse_scorer(model, Z_train, Z_test, y_train, y_test)
# KNN has good scores but takes orders of magnitude time longer (10+ minutes) so shouldn't
# implement for user side functionality
# SVM is also pretty long, and has the worst scores

R2 scores for <class 'sklearn.linear_model.base.LinearRegression'> train set: 0.7998428809318361
R2 scores for <class 'sklearn.linear_model.base.LinearRegression'> test set: -2.1657883903251046e+23
The training RMSE for <class 'sklearn.linear_model.base.LinearRegression'> is: 
10388.948010809852
The testing RMSE for <class 'sklearn.linear_model.base.LinearRegression'> is: 
1.0169723572506662e+16

R2 scores for <class 'sklearn.neighbors.regression.KNeighborsRegressor'> train set: 0.8791982969879253
R2 scores for <class 'sklearn.neighbors.regression.KNeighborsRegressor'> test set: 0.8440173262915125
The training RMSE for <class 'sklearn.neighbors.regression.KNeighborsRegressor'> is: 
8070.911415906978
The testing RMSE for <class 'sklearn.neighbors.regression.KNeighborsRegressor'> is: 
8630.56679047727

R2 scores for <class 'sklearn.tree.tree.DecisionTreeRegressor'> train set: 0.9621831940443599
R2 scores for <class 'sklearn.tree.tree.DecisionTreeRegressor'> test set: 0.5686861683209876
T

In [74]:
# Testing better models with PCA number of columns set to best 200 (out of 1482)
bagged_reg_PCA = BaggingRegressor()
bagged_reg_PCA.fit(Z_train, y_train)

adaboost_reg_PCA = AdaBoostRegressor()
adaboost_reg_PCA.fit(Z_train, y_train)

random_forest_reg_PCA = RandomForestRegressor()
random_forest_reg_PCA.fit(Z_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [75]:
for model in [bagged_reg_PCA, adaboost_reg_PCA, random_forest_reg_PCA] :
    rmse_scorer(model, Z_train, Z_test, y_train, y_test)
# restricting to top 200 variables of PCA axis rotation only improves r2 scores by about 1 percent here

R2 scores for <class 'sklearn.ensemble.bagging.BaggingRegressor'> train set: 0.931418388037298
R2 scores for <class 'sklearn.ensemble.bagging.BaggingRegressor'> test set: 0.7596213917622368
The training RMSE for <class 'sklearn.ensemble.bagging.BaggingRegressor'> is: 
0.24040429478189504
The testing RMSE for <class 'sklearn.ensemble.bagging.BaggingRegressor'> is: 
0.4284549107959935

R2 scores for <class 'sklearn.ensemble.weight_boosting.AdaBoostRegressor'> train set: 0.3431086962397566
R2 scores for <class 'sklearn.ensemble.weight_boosting.AdaBoostRegressor'> test set: 0.29172781641549006
The training RMSE for <class 'sklearn.ensemble.weight_boosting.AdaBoostRegressor'> is: 
0.7440208975927295
The testing RMSE for <class 'sklearn.ensemble.weight_boosting.AdaBoostRegressor'> is: 
0.7354570402582684

R2 scores for <class 'sklearn.ensemble.forest.RandomForestRegressor'> train set: 0.9305483221778924
R2 scores for <class 'sklearn.ensemble.forest.RandomForestRegressor'> test set: 0.7683522

### Testing 7 brackets of Income (0-11k, 11-21k, 21-31k, 31-41k, 41-51k, 51k-61k, 61k+)

In [21]:
print(y_allcities[y_allcities < 11000].shape)
print(y_allcities[(y_allcities >= 11000) & (y_allcities < 21000)].shape)
print(y_allcities[(y_allcities >= 21000) & (y_allcities < 31000)].shape)
print(y_allcities[(y_allcities >= 31000) & (y_allcities < 41000)].shape)
print(y_allcities[(y_allcities >= 41000) & (y_allcities < 51000)].shape)
print(y_allcities[(y_allcities >= 51000) & (y_allcities < 61000)].shape)
print(y_allcities[y_allcities >= 61000].shape)

(717,)
(3283,)
(1272,)
(883,)
(947,)
(524,)
(1876,)


In [58]:
## Creating column y_train_class

y_train = y_allcities

y_train_class = pd.Series(['NaN' for i in range(len(y_train)) ])

## Creating y_train for classification

brackets ={'$0-$10.99k': 0,'$11-$20.99k':1, '$21-$30.99k':2, '$31-$40.99k':3, '$41-$50.99k':4, 
           '$51-$60.99k':5,'$61-$70.99k':6, '$71+k':7}

for index, value in enumerate(y_train):
    if y_train.iloc[index] < 11000:
        y_train_class.iloc[index] = brackets['$0-$10.99k']
    elif y_train.iloc[index] < 21000:
        y_train_class.iloc[index] = brackets['$11-$20.99k']
    elif y_train.iloc[index] < 31000:
        y_train_class.iloc[index] = brackets['$21-$30.99k']
    elif y_train.iloc[index] < 41000:
        y_train_class.iloc[index] = brackets['$31-$40.99k']   
    elif y_train.iloc[index] < 51000:
        y_train_class.iloc[index] = brackets['$41-$50.99k']
    elif y_train.iloc[index] < 61000:
        y_train_class.iloc[index] = brackets['$51-$60.99k']
    elif y_train.iloc[index] < 71000:
        y_train_class.iloc[index] = brackets['$61-$70.99k']   
    elif y_train.iloc[index] >= 71000:
        y_train_class.iloc[index] = brackets['$71+k']
    else:
        y_train_class.iloc[index] = np.nan

In [123]:
X_train, X_test, y_train, y_test = train_test_split(X_allcities, y_train_class, 
                                                    test_size=.15, random_state=42) 
X_train.shape

(8076, 1482)

#### Logistic and Random Forest without PCA to get Theoretical Feature Importances

In [60]:
logreg_class_pca = LogisticRegression()
logreg_class_pca.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [124]:
random_forest_class_pca = RandomForestClassifier()
random_forest_class_pca.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [64]:
for model in [logreg_class_pca, random_forest_class_pca] :
    accuracy_scorer(model, X_train, X_test, y_train, y_test)

R2 scores for <class 'sklearn.linear_model.logistic.LogisticRegression'> train set: 0.8481921743437345
R2 scores for <class 'sklearn.linear_model.logistic.LogisticRegression'> test set: 0.7931276297335204
The training accuracy-score for <class 'sklearn.linear_model.logistic.LogisticRegression'> is: 
[0.67444327 0.90351165 0.77488372 0.7204142  0.86721504 0.88277268
 0.9375     0.79084552]
The testing accuracy-score for <class 'sklearn.linear_model.logistic.LogisticRegression'> is: 
[0.67647059 0.88435374 0.70801034 0.68217054 0.71942446 0.78688525
 0.88948787 0.67615658]

R2 scores for <class 'sklearn.ensemble.forest.RandomForestClassifier'> train set: 0.9972758791480931
R2 scores for <class 'sklearn.ensemble.forest.RandomForestClassifier'> test set: 0.923562412342216
The training accuracy-score for <class 'sklearn.ensemble.forest.RandomForestClassifier'> is: 
[0.99915326 0.99858005 0.9948861  0.99389831 0.99889503 0.99596774
 0.99818676 0.99626866]
The testing accuracy-score for <clas

In [65]:
joblib.dump(logreg_class_pca, 'log_No_pca_8brackets.joblib', compress=9)

joblib.dump(random_forest_class_pca, 'rf_No_pca_8brackets.joblib', compress=9)

['rf_No_pca_8brackets.joblib']

In [125]:
list(zip(random_forest_class_pca.feature_importances_, X_features))

[(0.0, 'landarea'),
 (0.0, 'waterarea'),
 (0.0038074042500468087, '0_restaurants_near'),
 (0.006734272856070376, '0_restaurants_far'),
 (0.0030858911375503673, '1_restaurants_near'),
 (0.00525303365963033, '1_restaurants_far'),
 (0.004165106293250027, '2_restaurants_near'),
 (0.0049108118574405415, '2_restaurants_far'),
 (0.0007339895288785583, '3_restaurants_near'),
 (0.001587224783825894, '3_restaurants_far'),
 (0.0001847675794587965, '4_restaurants_near'),
 (0.0007616234729181458, '4_restaurants_far'),
 (0.002460614395239728, '0_shopping_near'),
 (0.005839641361950303, '0_shopping_far'),
 (0.0014762140290383158, '1_shopping_near'),
 (0.0031315367612496244, '1_shopping_far'),
 (0.0025288289813820337, '2_shopping_near'),
 (0.003713365721191985, '2_shopping_far'),
 (0.0025325211481400036, '3_shopping_near'),
 (0.00281477503371727, '3_shopping_far'),
 (0.0025169458365963877, '4_shopping_near'),
 (0.001233903347374364, '4_shopping_far'),
 (0.002135747212591818, '0_food_near'),
 (0.004191

In [126]:
joblib.dump(random_forest_class_pca, 'rf_No_pca_8brackets_2ndcities.joblib', compress=9)

['rf_No_pca_8brackets_2ndcities.joblib']

In [137]:
pd.DataFrame(data=[X_features,list(map((lambda x : x[2:]),X_features))]).T.to_csv('temp.csv')

In [136]:
pd.DataFrame(data=list(zip(random_forest_class_pca.feature_importances_, X_features))).to_csv('temp.csv')

In [68]:
random_forest_class_pca_1stcities = joblib.load('rf_No_pca_8brackets_1stcities.joblib')

In [69]:
list(zip(random_forest_class_pca_1stcities.feature_importances_, X_features))

[(0.0, 'landarea'),
 (0.0, 'waterarea'),
 (0.0021095921790699776, '0_restaurants_near'),
 (0.0037967031904804368, '0_restaurants_far'),
 (0.004012371320830583, '1_restaurants_near'),
 (0.005897586992586921, '1_restaurants_far'),
 (0.003975995028779629, '2_restaurants_near'),
 (0.008046676599775291, '2_restaurants_far'),
 (0.0013134025714830056, '3_restaurants_near'),
 (0.0033572584892881223, '3_restaurants_far'),
 (0.0002879827447783404, '4_restaurants_near'),
 (0.0005052311527076426, '4_restaurants_far'),
 (0.0014060779121837128, '0_shopping_near'),
 (0.005717153089230735, '0_shopping_far'),
 (0.0017126138078900732, '1_shopping_near'),
 (0.0040018461782621246, '1_shopping_far'),
 (0.002715388290643132, '2_shopping_near'),
 (0.0073098835235059015, '2_shopping_far'),
 (0.0006078599219952206, '3_shopping_near'),
 (0.002943799738967007, '3_shopping_far'),
 (0.0005930018121901849, '4_shopping_near'),
 (0.0025480952571353063, '4_shopping_far'),
 (0.0015453170607575277, '0_food_near'),
 (0.0

In [ ]:
feature_importances = pd.DataFrame(columns=X_features)

In [102]:
feature_importances = feature_importances.append(pd.Series(data=random_forest_class_pca.feature_importances_, 
                                     name='rf_8brackets_2ndcities', index=X_features))
feature_importances

,landarea,waterarea,0_restaurants_near,0_restaurants_far,1_restaurants_near,1_restaurants_far,2_restaurants_near,2_restaurants_far,3_restaurants_near,3_restaurants_far,4_restaurants_near,4_restaurants_far,0_shopping_near,0_shopping_far,1_shopping_near,1_shopping_far,2_shopping_near,2_shopping_far,3_shopping_near,3_shopping_far,4_shopping_near,4_shopping_far,0_food_near,0_food_far,1_food_near,1_food_far,2_food_near,2_food_far,3_food_near,3_food_far,4_food_near,4_food_far,0_beautysvc_near,0_beautysvc_far,1_beautysvc_near,1_beautysvc_far,2_beautysvc_near,2_beautysvc_far,3_beautysvc_near,3_beautysvc_far,4_beautysvc_near,4_beautysvc_far,0_homeservices_near,0_homeservices_far,1_homeservices_near,...,2_hair_extensions_far,3_hair_extensions_near,3_hair_extensions_far,4_hair_extensions_near,4_hair_extensions_far,0_lawyers_near,0_lawyers_far,1_lawyers_near,1_lawyers_far,2_lawyers_near,2_lawyers_far,3_lawyers_near,3_lawyers_far,4_lawyers_near,4_lawyers_far,0_french_near,0_french_far,1_french_near,1_french_far,2_french_near,2_french_far,3_french_near,3_french_far,4_french_near,4_french_far,0_selfstorage_near,0_selfstorage_far,1_selfstorage_near,1_selfstorage_far,2_selfstorage_near,2_selfstorage_far,3_selfstorage_near,3_selfstorage_far,4_selfstorage_near,4_selfstorage_far,0_physicaltherapy_near,0_physicaltherapy_far,1_physicaltherapy_near,1_physicaltherapy_far,2_physicaltherapy_near,2_physicaltherapy_far,3_physicaltherapy_near,3_physicaltherapy_far,4_physicaltherapy_near,4_physicaltherapy_far
rf_8brackets_1stcities,0.0,0.0,0.002110,0.003797,0.004012,0.005898,0.003976,0.008047,0.001313,0.003357,0.000288,0.000505,0.001406,0.005717,0.001713,0.004002,0.002715,0.007310,0.000608,0.002944,0.000593,0.002548,0.001545,0.001932,0.001758,0.005012,0.002941,0.006851,0.000705,0.001483,0.000314,0.000202,0.000860,0.003304,0.001136,0.003939,0.001517,0.007499,0.000914,0.001661,0.000063,0.000816,0.004972,0.008929,0.00012,...,0.001050,0.00005,0.000113,0.0,0.0,0.000605,0.001603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000059,0.000000,0.000135,0.000343,0.000162,0.001269,0.000387,0.000342,0.000029,0.000426,0.000840,0.002445,0.0,0.0,0.000013,0.001330,0.0,0.0,0.0,0.0,0.000554,0.002445,0.0,0.000019,0.0,0.000368,0.0,0.0,0.0,0.0
rf_8brackets_2ndcities,0.0,0.0,0.003285,0.007479,0.003470,0.003876,0.001329,0.002676,0.000416,0.001662,0.000297,0.000723,0.002532,0.005679,0.000799,0.003040,0.002291,0.003421,0.001423,0.001308,0.000759,0.000778,0.002739,0.002216,0.003286,0.002208,0.001484,0.003362,0.001509,0.000596,0.000809,0.000303,0.001041,0.002030,0.001540,0.001832,0.001114,0.002306,0.018471,0.002370,0.000222,0.000164,0.002747,0.004325,0.00000,...,0.001305,0.00000,0.000000,0.0,0.0,0.001280,0.000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000860,0.000241,0.000000,0.000219,0.000254,0.000325,0.000320,0.001632,0.000000,0.000205,0.001934,0.001889,0.0,0.0,0.000000,0.000031,0.0,0.0,0.0,0.0,0.002659,0.013995,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [24]:

ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

## Applying PCA to Xs
pca = PCA(n_components=200)
pca = pca.fit(X_train_scaled)
Z_train = pca.transform(X_train_scaled)
Z_test =pca.transform(X_test_scaled)

In [25]:
logreg_class_pca = LogisticRegression()
logreg_class_pca.fit(Z_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
knn_class_pca = KNeighborsClassifier()
knn_class_pca.fit(Z_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [29]:
cart_class_pca = DecisionTreeClassifier()
cart_class_pca.fit(Z_train, y_train)

bagged_class_pca = BaggingClassifier()
bagged_class_pca.fit(Z_train, y_train)

BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
         verbose=0, warm_start=False)

In [30]:
random_forest_class_pca = RandomForestClassifier()
random_forest_class_pca.fit(Z_train, y_train)

adaboost_class_pca = AdaBoostClassifier()
adaboost_class_pca.fit(Z_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [31]:
for model in [logreg_class_pca, random_forest_class_pca, adaboost_class_pca] :
    accuracy_scorer(model, Z_train, Z_test, y_train, y_test)
# AdaBoost is the worst classifier among those tested, with default parameters, and
# with this level of income brackets

R2 scores for <class 'sklearn.linear_model.logistic.LogisticRegression'> train set: 0.7925953442298167
R2 scores for <class 'sklearn.linear_model.logistic.LogisticRegression'> test set: 0.7377279102384292
The training accuracy-score for <class 'sklearn.linear_model.logistic.LogisticRegression'> is: 
[0.68134172 0.85291339 0.67722165 0.66616541 0.79394645 0.7954071
 0.85925242 0.77284264]
The testing accuracy-score for <class 'sklearn.linear_model.logistic.LogisticRegression'> is: 
[0.625      0.82093664 0.63636364 0.65612648 0.64615385 0.73446328
 0.79234973 0.67148014]

R2 scores for <class 'sklearn.ensemble.forest.RandomForestClassifier'> train set: 0.9967805844477464
R2 scores for <class 'sklearn.ensemble.forest.RandomForestClassifier'> test set: 0.9025245441795231
The training accuracy-score for <class 'sklearn.ensemble.forest.RandomForestClassifier'> is: 
[0.9957663  0.99769135 0.9948861  0.9966239  0.99667774 0.9959596
 0.99773448 0.99626866]
The testing accuracy-score for <class

In [32]:
for model in [cart_class_pca, bagged_class_pca, knn_class_pca] :
    accuracy_scorer(model, Z_train, Z_test, y_train, y_test)

R2 scores for <class 'sklearn.tree.tree.DecisionTreeClassifier'> train set: 1.0
R2 scores for <class 'sklearn.tree.tree.DecisionTreeClassifier'> test set: 0.8660589060308556
The training accuracy-score for <class 'sklearn.tree.tree.DecisionTreeClassifier'> is: 
[1. 1. 1. 1. 1. 1. 1. 1.]
The testing accuracy-score for <class 'sklearn.tree.tree.DecisionTreeClassifier'> is: 
[0.89056604 0.93756614 0.85492228 0.79710145 0.74820144 0.7826087
 0.89130435 0.76816609]

R2 scores for <class 'sklearn.ensemble.bagging.BaggingClassifier'> train set: 0.9949232293214463
R2 scores for <class 'sklearn.ensemble.bagging.BaggingClassifier'> test set: 0.8962131837307152
The training accuracy-score for <class 'sklearn.ensemble.bagging.BaggingClassifier'> is: 
[0.99746407 0.99733901 0.99350046 0.98845893 0.99448732 0.99595142
 0.99317251 0.99442033]
The testing accuracy-score for <class 'sklearn.ensemble.bagging.BaggingClassifier'> is: 
[0.90980392 0.94409938 0.89405685 0.81318681 0.77464789 0.86813187
 0.9

In [33]:
joblib.dump(logreg_class_pca, 'log_pca_8brackets.joblib', compress=9)

joblib.dump(random_forest_class_pca, 'rf_pca_8brackets.joblib', compress=9)

['log_pca_8brackets.joblib']

### Testing 4 brackets of Income (0-20k, 20-40k, 40k-60k, 61k+)

In [85]:
print(y_allcities[y_allcities < 20000].shape)
print(y_allcities[(y_allcities >= 20000) & (y_allcities < 40000)].shape)
print(y_allcities[(y_allcities >= 40000) & (y_allcities < 60000)].shape)
print(y_allcities[y_allcities >= 60000].shape)

(3779,)
(2324,)
(1432,)
(1967,)


In [86]:
## Creating column y_train_class2

y_train = y_allcities

y_train_class2 = pd.Series(['NaN' for i in range(len(y_train)) ])

## Creating y_train for classification

brackets ={'$0-$20.99k': 0,'$21-$40.99k':1, '$41-$60.99k':2, '$61+k':3}

for index, value in enumerate(y_train):
    if y_train.iloc[index] < 21000:
        y_train_class2.iloc[index] = brackets['$0-$20.99k']
    elif y_train.iloc[index] < 41000:
        y_train_class2.iloc[index] = brackets['$21-$40.99k']
    elif y_train.iloc[index] < 61000:
        y_train_class2.iloc[index] = brackets['$41-$60.99k']
    elif y_train.iloc[index] >= 61000:
        y_train_class2.iloc[index] = brackets['$61+k']
    else:
        y_train_class2.iloc[index] = np.nan

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X_allcities, y_train_class2, 
                                                    test_size=.15, random_state=42) 
X_train.shape

(8076, 1482)

In [90]:
# Models without PCA to get feature importances for comparisons
logreg_class_pca = LogisticRegression()
logreg_class_pca.fit(X_train, y_train)

knn_class_pca = KNeighborsClassifier()
knn_class_pca.fit(X_train, y_train)

bagged_class_pca = BaggingClassifier()
bagged_class_pca.fit(X_train, y_train)

random_forest_class_pca = RandomForestClassifier()
random_forest_class_pca.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [91]:
for model in [logreg_class_pca, knn_class_pca, bagged_class_pca, random_forest_class_pca] :
    accuracy_scorer(model, X_train, X_test, y_train, y_test)

R2 scores for <class 'sklearn.linear_model.logistic.LogisticRegression'> train set: 0.8990837048043586
R2 scores for <class 'sklearn.linear_model.logistic.LogisticRegression'> test set: 0.8688639551192145
The training accuracy-score for <class 'sklearn.linear_model.logistic.LogisticRegression'> is: 
[0.92398815 0.79479016 0.87817048 0.97219619]
The testing accuracy-score for <class 'sklearn.linear_model.logistic.LogisticRegression'> is: 
[0.9138756  0.77020602 0.79807692 0.93284936]

R2 scores for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> train set: 0.9436602278355621
R2 scores for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> test set: 0.896914446002805
The training accuracy-score for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> is: 
[0.97353112 0.92337699 0.89705289 0.93831776]
The testing accuracy-score for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> is: 
[0.95973706 0.86544343 0.78873239 0.87927928]

R2 

In [103]:
feature_importances = feature_importances.append(pd.Series(data=random_forest_class_pca.feature_importances_, 
                                     name='rf_4brackets_2ndcities', index=X_features))
feature_importances

,landarea,waterarea,0_restaurants_near,0_restaurants_far,1_restaurants_near,1_restaurants_far,2_restaurants_near,2_restaurants_far,3_restaurants_near,3_restaurants_far,4_restaurants_near,4_restaurants_far,0_shopping_near,0_shopping_far,1_shopping_near,1_shopping_far,2_shopping_near,2_shopping_far,3_shopping_near,3_shopping_far,4_shopping_near,4_shopping_far,0_food_near,0_food_far,1_food_near,1_food_far,2_food_near,2_food_far,3_food_near,3_food_far,4_food_near,4_food_far,0_beautysvc_near,0_beautysvc_far,1_beautysvc_near,1_beautysvc_far,2_beautysvc_near,2_beautysvc_far,3_beautysvc_near,3_beautysvc_far,4_beautysvc_near,4_beautysvc_far,0_homeservices_near,0_homeservices_far,1_homeservices_near,...,2_hair_extensions_far,3_hair_extensions_near,3_hair_extensions_far,4_hair_extensions_near,4_hair_extensions_far,0_lawyers_near,0_lawyers_far,1_lawyers_near,1_lawyers_far,2_lawyers_near,2_lawyers_far,3_lawyers_near,3_lawyers_far,4_lawyers_near,4_lawyers_far,0_french_near,0_french_far,1_french_near,1_french_far,2_french_near,2_french_far,3_french_near,3_french_far,4_french_near,4_french_far,0_selfstorage_near,0_selfstorage_far,1_selfstorage_near,1_selfstorage_far,2_selfstorage_near,2_selfstorage_far,3_selfstorage_near,3_selfstorage_far,4_selfstorage_near,4_selfstorage_far,0_physicaltherapy_near,0_physicaltherapy_far,1_physicaltherapy_near,1_physicaltherapy_far,2_physicaltherapy_near,2_physicaltherapy_far,3_physicaltherapy_near,3_physicaltherapy_far,4_physicaltherapy_near,4_physicaltherapy_far
rf_8brackets_1stcities,0.0,0.0,0.002110,0.003797,0.004012,0.005898,0.003976,0.008047,0.001313,0.003357,0.000288,0.000505,0.001406,0.005717,0.001713,0.004002,0.002715,0.007310,0.000608,0.002944,0.000593,0.002548,0.001545,0.001932,0.001758,0.005012,0.002941,0.006851,0.000705,0.001483,0.000314,0.000202,0.000860,0.003304,0.001136,0.003939,0.001517,0.007499,0.000914,0.001661,0.000063,0.000816,0.004972,0.008929,0.00012,...,0.001050,0.00005,0.000113,0.0,0.0,0.000605,0.001603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000059,0.000000,0.000135,0.000343,0.000162,0.001269,0.000387,0.000342,0.000029,0.000426,0.000840,0.002445,0.0,0.0,0.000013,0.001330,0.0,0.0,0.0,0.0,0.000554,0.002445,0.0,0.000019,0.0,0.000368,0.0,0.0,0.0,0.0
rf_8brackets_2ndcities,0.0,0.0,0.003285,0.007479,0.003470,0.003876,0.001329,0.002676,0.000416,0.001662,0.000297,0.000723,0.002532,0.005679,0.000799,0.003040,0.002291,0.003421,0.001423,0.001308,0.000759,0.000778,0.002739,0.002216,0.003286,0.002208,0.001484,0.003362,0.001509,0.000596,0.000809,0.000303,0.001041,0.002030,0.001540,0.001832,0.001114,0.002306,0.018471,0.002370,0.000222,0.000164,0.002747,0.004325,0.00000,...,0.001305,0.00000,0.000000,0.0,0.0,0.001280,0.000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000860,0.000241,0.000000,0.000219,0.000254,0.000325,0.000320,0.001632,0.000000,0.000205,0.001934,0.001889,0.0,0.0,0.000000,0.000031,0.0,0.0,0.0,0.0,0.002659,0.013995,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
rf_4brackets_2ndcities,0.0,0.0,0.003285,0.007479,0.003470,0.003876,0.001329,0.002676,0.000416,0.001662,0.000297,0.000723,0.002532,0.005679,0.000799,0.003040,0.002291,0.003421,0.001423,0.001308,0.000759,0.000778,0.002739,0.002216,0.003286,0.002208,0.001484,0.003362,0.001509,0.000596,0.000809,0.000303,0.001041,0.002030,0.001540,0.001832,0.001114,0.002306,0.018471,0.002370,0.000222,0.000164,0.002747,0.004325,0.00000,...,0.001305,0.00000,0.000000,0.0,0.0,0.001280,0.000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000860,0.000241,0.000000,0.000219,0.000254,0.000325,0.000320,0.001632,0.000000,0.000205,0.001934,0.001889,0.0,0.0,0.000000,0.000031,0.0,0.0,0.0,0.0,0.002659,0.013995,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [104]:
for i in range(4) :
    feature_importances = feature_importances.append(pd.Series(data=logreg_class_pca.coef_[i], 
                                         name='log_4brackets_2ndcities_bracket'+str(i), index=X_features))
feature_importances

,landarea,waterarea,0_restaurants_near,0_restaurants_far,1_restaurants_near,1_restaurants_far,2_restaurants_near,2_restaurants_far,3_restaurants_near,3_restaurants_far,4_restaurants_near,4_restaurants_far,0_shopping_near,0_shopping_far,1_shopping_near,1_shopping_far,2_shopping_near,2_shopping_far,3_shopping_near,3_shopping_far,4_shopping_near,4_shopping_far,0_food_near,0_food_far,1_food_near,1_food_far,2_food_near,2_food_far,3_food_near,3_food_far,4_food_near,4_food_far,0_beautysvc_near,0_beautysvc_far,1_beautysvc_near,1_beautysvc_far,2_beautysvc_near,2_beautysvc_far,3_beautysvc_near,3_beautysvc_far,4_beautysvc_near,4_beautysvc_far,0_homeservices_near,0_homeservices_far,1_homeservices_near,...,2_hair_extensions_far,3_hair_extensions_near,3_hair_extensions_far,4_hair_extensions_near,4_hair_extensions_far,0_lawyers_near,0_lawyers_far,1_lawyers_near,1_lawyers_far,2_lawyers_near,2_lawyers_far,3_lawyers_near,3_lawyers_far,4_lawyers_near,4_lawyers_far,0_french_near,0_french_far,1_french_near,1_french_far,2_french_near,2_french_far,3_french_near,3_french_far,4_french_near,4_french_far,0_selfstorage_near,0_selfstorage_far,1_selfstorage_near,1_selfstorage_far,2_selfstorage_near,2_selfstorage_far,3_selfstorage_near,3_selfstorage_far,4_selfstorage_near,4_selfstorage_far,0_physicaltherapy_near,0_physicaltherapy_far,1_physicaltherapy_near,1_physicaltherapy_far,2_physicaltherapy_near,2_physicaltherapy_far,3_physicaltherapy_near,3_physicaltherapy_far,4_physicaltherapy_near,4_physicaltherapy_far
rf_8brackets_1stcities,0.0,0.0,0.002110,0.003797,0.004012,0.005898,0.003976,0.008047,0.001313,0.003357,0.000288,0.000505,0.001406,0.005717,0.001713,0.004002,0.002715,0.007310,0.000608,0.002944,0.000593,0.002548,0.001545,0.001932,0.001758,0.005012,0.002941,0.006851,0.000705,0.001483,0.000314,0.000202,0.000860,0.003304,0.001136,0.003939,0.001517,0.007499,0.000914,0.001661,0.000063,0.000816,0.004972,0.008929,0.000120,...,0.001050,0.000050,0.000113,0.000000,0.000000,0.000605,0.001603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000059,0.000000,0.000135,0.000343,0.000162,0.001269,0.000387,0.000342,0.000029,0.000426,0.000840,0.002445,0.0,0.000000,0.000013,0.001330,0.0,0.0,0.0,0.0,0.000554,0.002445,0.0,0.000019,0.000000,0.000368,0.0,0.0,0.0,0.0
rf_8brackets_2ndcities,0.0,0.0,0.003285,0.007479,0.003470,0.003876,0.001329,0.002676,0.000416,0.001662,0.000297,0.000723,0.002532,0.005679,0.000799,0.003040,0.002291,0.003421,0.001423,0.001308,0.000759,0.000778,0.002739,0.002216,0.003286,0.002208,0.001484,0.003362,0.001509,0.000596,0.000809,0.000303,0.001041,0.002030,0.001540,0.001832,0.001114,0.002306,0.018471,0.002370,0.000222,0.000164,0.002747,0.004325,0.000000,...,0.001305,0.000000,0.000000,0.000000,0.000000,0.001280,0.000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000860,0.000241,0.000000,0.000219,0.000254,0.000325,0.000320,0.001632,0.000000,0.000205,0.001934,0.001889,0.0,0.000000,0.000000,0.000031,0.0,0.0,0.0,0.0,0.002659,0.013995,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
rf_4brackets_2ndcities,0.0,0.0,0.003285,0.007479,0.003470,0.003876,0.001329,0.002676,0.000416,0.001662,0.000297,0.000723,0.002532,0.005679,0.000799,0.003040,0.002291,0.003421,0.001423,0.001308,0.000759,0.000778,0.002739,0.002216,0.003286,0.002208,0.001484,0.003362,0.001509,0.000596,0.000809,0.000303,0.001041,0.002030,0.001540,0.001832,0.001114,0.002306,0.018471,0.002370,0.000222,0.000164,0.002747,0.004325,0.000000,...,0.001305,0.000000,0.000000,0.000000,0.000000,0.001280,0.000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000860,0.000241,0.000000,0.000219,0.000254,0.000325,0.000320,0.001632,0.000000,0.000205,0.001934,0.001889,0.0,0.000000,0.000000,0.000031,0.0,0.0,0.0,0.0,0.002659,0.013995,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
log_4brackets_2ndcities_bracket0,0.0,0.0,0.084234,0.015501,0.054519,0.007493,-0.015347,0.008813,-0.020286,0.054885,0.005533,0.024269,0.006375,0.024820,0.039349,0.048546,-0.034534,-0.018928,-0.004315,-0.005306,-0.017773,0.001588,-0.003949,-0.010843,-0.032503,0.000222,0.008349,-0.00723

In [108]:
feature_importances = feature_importances.append(pd.Series(data=X_allcities.sum(), 
                                     name='2ndcities_category_counts', index=X_features))
feature_importances

,landarea,waterarea,0_restaurants_near,0_restaurants_far,1_restaurants_near,1_restaurants_far,2_restaurants_near,2_restaurants_far,3_restaurants_near,3_restaurants_far,4_restaurants_near,4_restaurants_far,0_shopping_near,0_shopping_far,1_shopping_near,1_shopping_far,2_shopping_near,2_shopping_far,3_shopping_near,3_shopping_far,4_shopping_near,4_shopping_far,0_food_near,0_food_far,1_food_near,1_food_far,2_food_near,2_food_far,3_food_near,3_food_far,4_food_near,4_food_far,0_beautysvc_near,0_beautysvc_far,1_beautysvc_near,1_beautysvc_far,2_beautysvc_near,2_beautysvc_far,3_beautysvc_near,3_beautysvc_far,4_beautysvc_near,4_beautysvc_far,0_homeservices_near,0_homeservices_far,1_homeservices_near,...,2_hair_extensions_far,3_hair_extensions_near,3_hair_extensions_far,4_hair_extensions_near,4_hair_extensions_far,0_lawyers_near,0_lawyers_far,1_lawyers_near,1_lawyers_far,2_lawyers_near,2_lawyers_far,3_lawyers_near,3_lawyers_far,4_lawyers_near,4_lawyers_far,0_french_near,0_french_far,1_french_near,1_french_far,2_french_near,2_french_far,3_french_near,3_french_far,4_french_near,4_french_far,0_selfstorage_near,0_selfstorage_far,1_selfstorage_near,1_selfstorage_far,2_selfstorage_near,2_selfstorage_far,3_selfstorage_near,3_selfstorage_far,4_selfstorage_near,4_selfstorage_far,0_physicaltherapy_near,0_physicaltherapy_far,1_physicaltherapy_near,1_physicaltherapy_far,2_physicaltherapy_near,2_physicaltherapy_far,3_physicaltherapy_near,3_physicaltherapy_far,4_physicaltherapy_near,4_physicaltherapy_far
rf_8brackets_1stcities,0.0,0.0,0.002110,0.003797,0.004012,0.005898,0.003976,0.008047,0.001313,0.003357,0.000288,0.000505,0.001406,5.717153e-03,0.001713,0.004002,0.002715,7.309884e-03,0.000608,0.002944,0.000593,0.002548,0.001545,1.931834e-03,0.001758,0.005012,0.002941,0.006851,0.000705,0.001483,0.000314,0.000202,0.000860,3.303725e-03,0.001136,0.003939,0.001517,7.499336e-03,0.000914,0.001661,0.000063,0.000816,4.972478e-03,8.929085e-03,0.000120,...,0.001050,0.000050,0.000113,0.000000,0.000000,0.000605,0.001603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000059,0.000000,0.000135,0.000343,0.000162,0.001269,0.000387,0.000342,0.000029,0.000426,0.000840,0.002445,0.0,0.000000,0.000013,0.001330,0.0,0.0,0.0,0.0,0.000554,0.002445,0.0,0.000019,0.000000,0.000368,0.0,0.0,0.0,0.0
rf_8brackets_2ndcities,0.0,0.0,0.003285,0.007479,0.003470,0.003876,0.001329,0.002676,0.000416,0.001662,0.000297,0.000723,0.002532,5.679266e-03,0.000799,0.003040,0.002291,3.420709e-03,0.001423,0.001308,0.000759,0.000778,0.002739,2.215691e-03,0.003286,0.002208,0.001484,0.003362,0.001509,0.000596,0.000809,0.000303,0.001041,2.029535e-03,0.001540,0.001832,0.001114,2.305622e-03,0.018471,0.002370,0.000222,0.000164,2.747113e-03,4.324658e-03,0.000000,...,0.001305,0.000000,0.000000,0.000000,0.000000,0.001280,0.000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000860,0.000241,0.000000,0.000219,0.000254,0.000325,0.000320,0.001632,0.000000,0.000205,0.001934,0.001889,0.0,0.000000,0.000000,0.000031,0.0,0.0,0.0,0.0,0.002659,0.013995,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
rf_4brackets_2ndcities,0.0,0.0,0.003285,0.007479,0.003470,0.003876,0.001329,0.002676,0.000416,0.001662,0.000297,0.000723,0.002532,5.679266e-03,0.000799,0.003040,0.002291,3.420709e-03,0.001423,0.001308,0.000759,0.000778,0.002739,2.215691e-03,0.003286,0.002208,0.001484,0.003362,0.001509,0.000596,0.000809,0.000303,0.001041,2.029535e-03,0.001540,0.001832,0.001114,2.305622e-03,0.018471,0.002370,0.000222,0.000164,2.747113e-03,4.324658e-03,0.000000,...,0.001305,0.000000,0.000000,0.000000,0.000000,0.001280,0.000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000860,0.000241,0.000000,0.000219,0.000254,0.000325,0.000320,0.001632,0.000000,0.000205,0.001934,0.001889,0.0,0.000000,0.000000,0.000031,0.0,0.0,0.0,0.0,0.002659,0.013995,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
log_4brackets_2ndcities_bracket0,0.0,0.0,0.084234,0.015501,0.054519,0.007493,-0.015347,0.008813,-0.020286,0.054885,0.005533,0.024269,0.006375,2.481977e-02,0.039349,0.048546,-0.034534,-1.892845e-02,-0.

In [112]:
temp1 = joblib.load('1stcities_category_counts.joblib')

temp2 = joblib.load('rf_No_pca_4brackets_1stcities.joblib')

temp3 = joblib.load('log_No_pca_4brackets_1stcities.joblib')

In [113]:
feature_importances = feature_importances.append(pd.Series(data=temp1, 
                                     name='1stcities_category_counts', index=X_features))
feature_importances

,landarea,waterarea,0_restaurants_near,0_restaurants_far,1_restaurants_near,1_restaurants_far,2_restaurants_near,2_restaurants_far,3_restaurants_near,3_restaurants_far,4_restaurants_near,4_restaurants_far,0_shopping_near,0_shopping_far,1_shopping_near,1_shopping_far,2_shopping_near,2_shopping_far,3_shopping_near,3_shopping_far,4_shopping_near,4_shopping_far,0_food_near,0_food_far,1_food_near,1_food_far,2_food_near,2_food_far,3_food_near,3_food_far,4_food_near,4_food_far,0_beautysvc_near,0_beautysvc_far,1_beautysvc_near,1_beautysvc_far,2_beautysvc_near,2_beautysvc_far,3_beautysvc_near,3_beautysvc_far,4_beautysvc_near,4_beautysvc_far,0_homeservices_near,0_homeservices_far,1_homeservices_near,...,2_hair_extensions_far,3_hair_extensions_near,3_hair_extensions_far,4_hair_extensions_near,4_hair_extensions_far,0_lawyers_near,0_lawyers_far,1_lawyers_near,1_lawyers_far,2_lawyers_near,2_lawyers_far,3_lawyers_near,3_lawyers_far,4_lawyers_near,4_lawyers_far,0_french_near,0_french_far,1_french_near,1_french_far,2_french_near,2_french_far,3_french_near,3_french_far,4_french_near,4_french_far,0_selfstorage_near,0_selfstorage_far,1_selfstorage_near,1_selfstorage_far,2_selfstorage_near,2_selfstorage_far,3_selfstorage_near,3_selfstorage_far,4_selfstorage_near,4_selfstorage_far,0_physicaltherapy_near,0_physicaltherapy_far,1_physicaltherapy_near,1_physicaltherapy_far,2_physicaltherapy_near,2_physicaltherapy_far,3_physicaltherapy_near,3_physicaltherapy_far,4_physicaltherapy_near,4_physicaltherapy_far
rf_8brackets_2ndcities,0.0,0.0,0.003285,0.007479,0.003470,0.003876,0.001329,0.002676,0.000416,0.001662,0.000297,0.000723,0.002532,5.679266e-03,0.000799,0.003040,0.002291,3.420709e-03,0.001423,0.001308,0.000759,0.000778,0.002739,2.215691e-03,0.003286,0.002208,0.001484,0.003362,0.001509,0.000596,0.000809,0.000303,0.001041,2.029535e-03,0.001540,0.001832,0.001114,2.305622e-03,0.018471,0.002370,0.000222,0.000164,2.747113e-03,4.324658e-03,0.000000,...,0.001305,0.000000,0.000000,0.000000,0.000000,0.001280,0.000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000860,0.000241,0.000000,0.000219,0.000254,0.000325,0.000320,0.001632,0.000000,0.000205,0.001934,0.001889,0.0,0.000000,0.000000,0.000031,0.0,0.0,0.0,0.0,0.002659,0.013995,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
rf_4brackets_2ndcities,0.0,0.0,0.003285,0.007479,0.003470,0.003876,0.001329,0.002676,0.000416,0.001662,0.000297,0.000723,0.002532,5.679266e-03,0.000799,0.003040,0.002291,3.420709e-03,0.001423,0.001308,0.000759,0.000778,0.002739,2.215691e-03,0.003286,0.002208,0.001484,0.003362,0.001509,0.000596,0.000809,0.000303,0.001041,2.029535e-03,0.001540,0.001832,0.001114,2.305622e-03,0.018471,0.002370,0.000222,0.000164,2.747113e-03,4.324658e-03,0.000000,...,0.001305,0.000000,0.000000,0.000000,0.000000,0.001280,0.000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000860,0.000241,0.000000,0.000219,0.000254,0.000325,0.000320,0.001632,0.000000,0.000205,0.001934,0.001889,0.0,0.000000,0.000000,0.000031,0.0,0.0,0.0,0.0,0.002659,0.013995,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
log_4brackets_2ndcities_bracket0,0.0,0.0,0.084234,0.015501,0.054519,0.007493,-0.015347,0.008813,-0.020286,0.054885,0.005533,0.024269,0.006375,2.481977e-02,0.039349,0.048546,-0.034534,-1.892845e-02,-0.004315,-0.005306,-0.017773,0.001588,-0.003949,-1.084340e-02,-0.032503,0.000222,0.008349,-0.007234,-0.004934,0.034189,-0.001773,0.016933,0.030312,8.406059e-03,-0.007191,-0.005327,-0.002194,2.901784e-03,0.003973,-0.003518,-0.005228,0.026607,7.183351e-03,1.011133e-02,-0.000649,...,0.023344,-0.004179,0.005668,-0.003517,0.005685,0.014489,0.001650,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.005724,-0.009817,-0.002757,-0.006627,-0.000829,0.002133,-0.002278,-0.014689,-0.000713,0.000323,-0.002625,0.036626,0.0,-0.000022,0.000207,0.004663,0.0,0.0,0.0,0.0,0.002469,-0.035264,0.0,0.0,-0.001230,-0.004102,0.0,0.0,0.0,0.0
log_4brackets_2ndcities_bracket1,0.0,0.0,-0.064469,-0.026284,0.005846,0.004611,0.028361,0.013414,-0.003971,-0.024274,-0.005943,-0.010671,0.010697,-1.827375e-02,-0.003921,-

In [115]:
temp2.feature_importances_.shape

(1482,)

In [116]:
feature_importances = feature_importances.append(pd.Series(data=temp2.feature_importances_, 
                                     name='rf_4brackets_1stcities', index=X_features))
feature_importances

,landarea,waterarea,0_restaurants_near,0_restaurants_far,1_restaurants_near,1_restaurants_far,2_restaurants_near,2_restaurants_far,3_restaurants_near,3_restaurants_far,4_restaurants_near,4_restaurants_far,0_shopping_near,0_shopping_far,1_shopping_near,1_shopping_far,2_shopping_near,2_shopping_far,3_shopping_near,3_shopping_far,4_shopping_near,4_shopping_far,0_food_near,0_food_far,1_food_near,1_food_far,2_food_near,2_food_far,3_food_near,3_food_far,4_food_near,4_food_far,0_beautysvc_near,0_beautysvc_far,1_beautysvc_near,1_beautysvc_far,2_beautysvc_near,2_beautysvc_far,3_beautysvc_near,3_beautysvc_far,4_beautysvc_near,4_beautysvc_far,0_homeservices_near,0_homeservices_far,1_homeservices_near,...,2_hair_extensions_far,3_hair_extensions_near,3_hair_extensions_far,4_hair_extensions_near,4_hair_extensions_far,0_lawyers_near,0_lawyers_far,1_lawyers_near,1_lawyers_far,2_lawyers_near,2_lawyers_far,3_lawyers_near,3_lawyers_far,4_lawyers_near,4_lawyers_far,0_french_near,0_french_far,1_french_near,1_french_far,2_french_near,2_french_far,3_french_near,3_french_far,4_french_near,4_french_far,0_selfstorage_near,0_selfstorage_far,1_selfstorage_near,1_selfstorage_far,2_selfstorage_near,2_selfstorage_far,3_selfstorage_near,3_selfstorage_far,4_selfstorage_near,4_selfstorage_far,0_physicaltherapy_near,0_physicaltherapy_far,1_physicaltherapy_near,1_physicaltherapy_far,2_physicaltherapy_near,2_physicaltherapy_far,3_physicaltherapy_near,3_physicaltherapy_far,4_physicaltherapy_near,4_physicaltherapy_far
rf_8brackets_2ndcities,0.0,0.0,0.003285,0.007479,0.003470,0.003876,0.001329,0.002676,0.000416,0.001662,0.000297,0.000723,0.002532,5.679266e-03,0.000799,0.003040,0.002291,3.420709e-03,0.001423,0.001308,0.000759,0.000778,0.002739,2.215691e-03,0.003286,0.002208,0.001484,0.003362,0.001509,0.000596,0.000809,0.000303,0.001041,2.029535e-03,0.001540,0.001832,0.001114,2.305622e-03,0.018471,0.002370,0.000222,0.000164,2.747113e-03,4.324658e-03,0.000000,...,0.001305,0.000000,0.000000,0.000000,0.000000,0.001280,0.000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000860,0.000241,0.000000,0.000219,0.000254,0.000325,0.000320,0.001632,0.000000,0.000205,0.001934,0.001889,0.0,0.000000,0.000000,0.000031,0.000000,0.0,0.0,0.000000,0.002659,0.013995,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
rf_4brackets_2ndcities,0.0,0.0,0.003285,0.007479,0.003470,0.003876,0.001329,0.002676,0.000416,0.001662,0.000297,0.000723,0.002532,5.679266e-03,0.000799,0.003040,0.002291,3.420709e-03,0.001423,0.001308,0.000759,0.000778,0.002739,2.215691e-03,0.003286,0.002208,0.001484,0.003362,0.001509,0.000596,0.000809,0.000303,0.001041,2.029535e-03,0.001540,0.001832,0.001114,2.305622e-03,0.018471,0.002370,0.000222,0.000164,2.747113e-03,4.324658e-03,0.000000,...,0.001305,0.000000,0.000000,0.000000,0.000000,0.001280,0.000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000860,0.000241,0.000000,0.000219,0.000254,0.000325,0.000320,0.001632,0.000000,0.000205,0.001934,0.001889,0.0,0.000000,0.000000,0.000031,0.000000,0.0,0.0,0.000000,0.002659,0.013995,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
log_4brackets_2ndcities_bracket0,0.0,0.0,0.084234,0.015501,0.054519,0.007493,-0.015347,0.008813,-0.020286,0.054885,0.005533,0.024269,0.006375,2.481977e-02,0.039349,0.048546,-0.034534,-1.892845e-02,-0.004315,-0.005306,-0.017773,0.001588,-0.003949,-1.084340e-02,-0.032503,0.000222,0.008349,-0.007234,-0.004934,0.034189,-0.001773,0.016933,0.030312,8.406059e-03,-0.007191,-0.005327,-0.002194,2.901784e-03,0.003973,-0.003518,-0.005228,0.026607,7.183351e-03,1.011133e-02,-0.000649,...,0.023344,-0.004179,0.005668,-0.003517,0.005685,0.014489,0.001650,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.005724,-0.009817,-0.002757,-0.006627,-0.000829,0.002133,-0.002278,-0.014689,-0.000713,0.000323,-0.002625,0.036626,0.0,-0.000022,0.000207,0.004663,0.000000,0.0,0.0,0.000000,0.002469,-0.035264,0.0,0.000000,-0.001230,-0.004102,0.0,0.0,0.0,0.0
log_4brackets_2ndcities_bracket1,0.0,0.0,-0.064469,-0.026284,0.005846,0.004611,0.028361,0.013414,-0.003971,-0.024274,-0.005943

In [117]:
for i in range(4) :
    feature_importances = feature_importances.append(pd.Series(data=temp3.coef_[i], 
                                         name='log_4brackets_1stcities_bracket'+str(i), index=X_features))
feature_importances

,landarea,waterarea,0_restaurants_near,0_restaurants_far,1_restaurants_near,1_restaurants_far,2_restaurants_near,2_restaurants_far,3_restaurants_near,3_restaurants_far,4_restaurants_near,4_restaurants_far,0_shopping_near,0_shopping_far,1_shopping_near,1_shopping_far,2_shopping_near,2_shopping_far,3_shopping_near,3_shopping_far,4_shopping_near,4_shopping_far,0_food_near,0_food_far,1_food_near,1_food_far,2_food_near,2_food_far,3_food_near,3_food_far,4_food_near,4_food_far,0_beautysvc_near,0_beautysvc_far,1_beautysvc_near,1_beautysvc_far,2_beautysvc_near,2_beautysvc_far,3_beautysvc_near,3_beautysvc_far,4_beautysvc_near,4_beautysvc_far,0_homeservices_near,0_homeservices_far,1_homeservices_near,...,2_hair_extensions_far,3_hair_extensions_near,3_hair_extensions_far,4_hair_extensions_near,4_hair_extensions_far,0_lawyers_near,0_lawyers_far,1_lawyers_near,1_lawyers_far,2_lawyers_near,2_lawyers_far,3_lawyers_near,3_lawyers_far,4_lawyers_near,4_lawyers_far,0_french_near,0_french_far,1_french_near,1_french_far,2_french_near,2_french_far,3_french_near,3_french_far,4_french_near,4_french_far,0_selfstorage_near,0_selfstorage_far,1_selfstorage_near,1_selfstorage_far,2_selfstorage_near,2_selfstorage_far,3_selfstorage_near,3_selfstorage_far,4_selfstorage_near,4_selfstorage_far,0_physicaltherapy_near,0_physicaltherapy_far,1_physicaltherapy_near,1_physicaltherapy_far,2_physicaltherapy_near,2_physicaltherapy_far,3_physicaltherapy_near,3_physicaltherapy_far,4_physicaltherapy_near,4_physicaltherapy_far
rf_8brackets_2ndcities,0.0,0.0,0.003285,0.007479,0.003470,0.003876,0.001329,0.002676,0.000416,0.001662,0.000297,0.000723,0.002532,5.679266e-03,0.000799,0.003040,0.002291,3.420709e-03,0.001423,0.001308,0.000759,0.000778,0.002739,2.215691e-03,0.003286,0.002208,0.001484,0.003362,0.001509,0.000596,0.000809,0.000303,0.001041,2.029535e-03,0.001540,0.001832,0.001114,2.305622e-03,0.018471,0.002370,0.000222,0.000164,2.747113e-03,4.324658e-03,0.000000,...,0.001305,0.000000,0.000000,0.000000,0.000000,0.001280,0.000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000860,0.000241,0.000000,0.000219,0.000254,0.000325,0.000320,0.001632,0.000000,0.000205,0.001934,0.001889,0.000000,0.000000,0.000000,0.000031,0.000000,0.000000,0.000000,0.000000,0.002659,0.013995,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
rf_4brackets_2ndcities,0.0,0.0,0.003285,0.007479,0.003470,0.003876,0.001329,0.002676,0.000416,0.001662,0.000297,0.000723,0.002532,5.679266e-03,0.000799,0.003040,0.002291,3.420709e-03,0.001423,0.001308,0.000759,0.000778,0.002739,2.215691e-03,0.003286,0.002208,0.001484,0.003362,0.001509,0.000596,0.000809,0.000303,0.001041,2.029535e-03,0.001540,0.001832,0.001114,2.305622e-03,0.018471,0.002370,0.000222,0.000164,2.747113e-03,4.324658e-03,0.000000,...,0.001305,0.000000,0.000000,0.000000,0.000000,0.001280,0.000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000860,0.000241,0.000000,0.000219,0.000254,0.000325,0.000320,0.001632,0.000000,0.000205,0.001934,0.001889,0.000000,0.000000,0.000000,0.000031,0.000000,0.000000,0.000000,0.000000,0.002659,0.013995,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
log_4brackets_2ndcities_bracket0,0.0,0.0,0.084234,0.015501,0.054519,0.007493,-0.015347,0.008813,-0.020286,0.054885,0.005533,0.024269,0.006375,2.481977e-02,0.039349,0.048546,-0.034534,-1.892845e-02,-0.004315,-0.005306,-0.017773,0.001588,-0.003949,-1.084340e-02,-0.032503,0.000222,0.008349,-0.007234,-0.004934,0.034189,-0.001773,0.016933,0.030312,8.406059e-03,-0.007191,-0.005327,-0.002194,2.901784e-03,0.003973,-0.003518,-0.005228,0.026607,7.183351e-03,1.011133e-02,-0.000649,...,0.023344,-0.004179,0.005668,-0.003517,0.005685,0.014489,0.001650,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.005724,-0.009817,-0.002757,-0.006627,-0.000829,0.002133,-0.002278,-0.014689,-0.000713,0.000323,-0.002625,0.036626,0.000000,-0.000022,0.000207,0.004663,0.000000,0.000000,0.000000,0.000000,0.002469,-0.035264,0.000000,0.000000,-0.001230,-0.004102,0.0,0.0,0.0,0.0
log_4brackets_2ndcities_bracket1,0.0,0.0,-0.064469,-0.026284,0.005

In [118]:
feature_importances = feature_importances.append(pd.Series(data=random_forest_class_pca_1stcities.feature_importances_, 
                                     name='rf_8brackets_1stcities', index=X_features))

In [119]:
feature_importances

,landarea,waterarea,0_restaurants_near,0_restaurants_far,1_restaurants_near,1_restaurants_far,2_restaurants_near,2_restaurants_far,3_restaurants_near,3_restaurants_far,4_restaurants_near,4_restaurants_far,0_shopping_near,0_shopping_far,1_shopping_near,1_shopping_far,2_shopping_near,2_shopping_far,3_shopping_near,3_shopping_far,4_shopping_near,4_shopping_far,0_food_near,0_food_far,1_food_near,1_food_far,2_food_near,2_food_far,3_food_near,3_food_far,4_food_near,4_food_far,0_beautysvc_near,0_beautysvc_far,1_beautysvc_near,1_beautysvc_far,2_beautysvc_near,2_beautysvc_far,3_beautysvc_near,3_beautysvc_far,4_beautysvc_near,4_beautysvc_far,0_homeservices_near,0_homeservices_far,1_homeservices_near,...,2_hair_extensions_far,3_hair_extensions_near,3_hair_extensions_far,4_hair_extensions_near,4_hair_extensions_far,0_lawyers_near,0_lawyers_far,1_lawyers_near,1_lawyers_far,2_lawyers_near,2_lawyers_far,3_lawyers_near,3_lawyers_far,4_lawyers_near,4_lawyers_far,0_french_near,0_french_far,1_french_near,1_french_far,2_french_near,2_french_far,3_french_near,3_french_far,4_french_near,4_french_far,0_selfstorage_near,0_selfstorage_far,1_selfstorage_near,1_selfstorage_far,2_selfstorage_near,2_selfstorage_far,3_selfstorage_near,3_selfstorage_far,4_selfstorage_near,4_selfstorage_far,0_physicaltherapy_near,0_physicaltherapy_far,1_physicaltherapy_near,1_physicaltherapy_far,2_physicaltherapy_near,2_physicaltherapy_far,3_physicaltherapy_near,3_physicaltherapy_far,4_physicaltherapy_near,4_physicaltherapy_far
rf_8brackets_2ndcities,0.0,0.0,0.003285,0.007479,0.003470,0.003876,0.001329,0.002676,0.000416,0.001662,0.000297,0.000723,0.002532,5.679266e-03,0.000799,0.003040,0.002291,3.420709e-03,0.001423,0.001308,0.000759,0.000778,0.002739,2.215691e-03,0.003286,0.002208,0.001484,0.003362,0.001509,0.000596,0.000809,0.000303,0.001041,2.029535e-03,0.001540,0.001832,0.001114,2.305622e-03,0.018471,0.002370,0.000222,0.000164,2.747113e-03,4.324658e-03,0.000000,...,0.001305,0.000000,0.000000,0.000000,0.000000,0.001280,0.000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000860,0.000241,0.000000,0.000219,0.000254,0.000325,0.000320,0.001632,0.000000,0.000205,0.001934,0.001889,0.000000,0.000000,0.000000,0.000031,0.000000,0.000000,0.000000,0.000000,0.002659,0.013995,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
rf_4brackets_2ndcities,0.0,0.0,0.003285,0.007479,0.003470,0.003876,0.001329,0.002676,0.000416,0.001662,0.000297,0.000723,0.002532,5.679266e-03,0.000799,0.003040,0.002291,3.420709e-03,0.001423,0.001308,0.000759,0.000778,0.002739,2.215691e-03,0.003286,0.002208,0.001484,0.003362,0.001509,0.000596,0.000809,0.000303,0.001041,2.029535e-03,0.001540,0.001832,0.001114,2.305622e-03,0.018471,0.002370,0.000222,0.000164,2.747113e-03,4.324658e-03,0.000000,...,0.001305,0.000000,0.000000,0.000000,0.000000,0.001280,0.000508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000860,0.000241,0.000000,0.000219,0.000254,0.000325,0.000320,0.001632,0.000000,0.000205,0.001934,0.001889,0.000000,0.000000,0.000000,0.000031,0.000000,0.000000,0.000000,0.000000,0.002659,0.013995,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
log_4brackets_2ndcities_bracket0,0.0,0.0,0.084234,0.015501,0.054519,0.007493,-0.015347,0.008813,-0.020286,0.054885,0.005533,0.024269,0.006375,2.481977e-02,0.039349,0.048546,-0.034534,-1.892845e-02,-0.004315,-0.005306,-0.017773,0.001588,-0.003949,-1.084340e-02,-0.032503,0.000222,0.008349,-0.007234,-0.004934,0.034189,-0.001773,0.016933,0.030312,8.406059e-03,-0.007191,-0.005327,-0.002194,2.901784e-03,0.003973,-0.003518,-0.005228,0.026607,7.183351e-03,1.011133e-02,-0.000649,...,0.023344,-0.004179,0.005668,-0.003517,0.005685,0.014489,0.001650,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.005724,-0.009817,-0.002757,-0.006627,-0.000829,0.002133,-0.002278,-0.014689,-0.000713,0.000323,-0.002625,0.036626,0.000000,-0.000022,0.000207,0.004663,0.000000,0.000000,0.000000,0.000000,0.002469,-0.035264,0.000000,0.000000,-0.001230,-0.004102,0.0,0.0,0.0,0.0
log_4brackets_2ndcities_bracket1,0.0,0.0,-0.064469,-0.026284,0.005

In [120]:
joblib.dump(feature_importances, 'feature_importances_allDatasets.joblib', compress=9)

['feature_importances_allDatasets.joblib']

In [122]:
feature_importances.T.to_csv('feature_importances_allDatasets.csv')

In [38]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

## Applying PCA to Xs
pca = PCA(n_components=200)
pca = pca.fit(X_train_scaled)
Z_train = pca.transform(X_train_scaled)
Z_test =pca.transform(X_test_scaled)

In [39]:
logreg_class_pca = LogisticRegression()
logreg_class_pca.fit(Z_train, y_train)

knn_class_pca = KNeighborsClassifier()
knn_class_pca.fit(Z_train, y_train)

bagged_class_pca = BaggingClassifier()
bagged_class_pca.fit(Z_train, y_train)

random_forest_class_pca = RandomForestClassifier()
random_forest_class_pca.fit(Z_train, y_train)

adaboost_class_pca = AdaBoostClassifier()
adaboost_class_pca.fit(Z_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [40]:
for model in [logreg_class_pca, knn_class_pca, bagged_class_pca, random_forest_class_pca, 
              adaboost_class_pca] :
    accuracy_scorer(model, Z_train, Z_test, y_train, y_test)

R2 scores for <class 'sklearn.linear_model.logistic.LogisticRegression'> train set: 0.8311045071817732
R2 scores for <class 'sklearn.linear_model.logistic.LogisticRegression'> test set: 0.8022440392706872
The training accuracy-score for <class 'sklearn.linear_model.logistic.LogisticRegression'> is: 
[0.90196078 0.72033146 0.73322148 0.86113759]
The testing accuracy-score for <class 'sklearn.linear_model.logistic.LogisticRegression'> is: 
[0.88562597 0.69811321 0.65789474 0.82656827]

R2 scores for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> train set: 0.9408122833085686
R2 scores for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> test set: 0.8856942496493688
The training accuracy-score for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> is: 
[0.97005988 0.91507607 0.9006781  0.93873704]
The testing accuracy-score for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> is: 
[0.94137077 0.83692308 0.78995434 0.89692586]

R2

### Testing 3 brackets of Income (0-30k, 30k-60k, 61k+)

In [41]:
print(y_allcities[y_allcities < 30000].shape)
print(y_allcities[(y_allcities >= 30000) & (y_allcities < 60000)].shape)
print(y_allcities[y_allcities >= 60000].shape)

(5185,)
(2350,)
(1967,)


In [42]:
## Creating column y_train_class3

y_train = y_allcities

y_train_class3 = pd.Series(['NaN' for i in range(len(y_train)) ])

## Creating y_train for classification

brackets ={'$0-$30.99k': 0,'$31-$60.99k':1, '$61+k':3}

for index, value in enumerate(y_train):
    if y_train.iloc[index] < 31000:
        y_train_class3.iloc[index] = brackets['$0-$30.99k']
    elif y_train.iloc[index] < 61000:
        y_train_class3.iloc[index] = brackets['$31-$60.99k']
    elif y_train.iloc[index] >= 61000:
        y_train_class3.iloc[index] = brackets['$61+k']
    else:
        y_train_class3.iloc[index] = np.nan

In [43]:
# Also with stratifying on income
X_train, X_test, y_train, y_test = train_test_split(X_allcities, y_train_class3, 
                                                    test_size=.15, random_state=42,
                                                   stratify=y_train_class3) 
X_train.shape

ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

## Applying PCA to Xs
pca = PCA(n_components=200)
pca = pca.fit(X_train_scaled)
Z_train = pca.transform(X_train_scaled)
Z_test =pca.transform(X_test_scaled)

In [44]:
logreg_class_pca = LogisticRegression()
logreg_class_pca.fit(Z_train, y_train)

knn_class_pca = KNeighborsClassifier()
knn_class_pca.fit(Z_train, y_train)

bagged_class_pca = BaggingClassifier()
bagged_class_pca.fit(Z_train, y_train)

random_forest_class_pca = RandomForestClassifier()
random_forest_class_pca.fit(Z_train, y_train)

adaboost_class_pca = AdaBoostClassifier()
adaboost_class_pca.fit(Z_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [45]:
for model in [logreg_class_pca, knn_class_pca, bagged_class_pca, random_forest_class_pca, 
              adaboost_class_pca] :
    accuracy_scorer(model, Z_train, Z_test, y_train, y_test)

R2 scores for <class 'sklearn.linear_model.logistic.LogisticRegression'> train set: 0.8749380881624567
R2 scores for <class 'sklearn.linear_model.logistic.LogisticRegression'> test set: 0.8576437587657784
The training accuracy-score for <class 'sklearn.linear_model.logistic.LogisticRegression'> is: 
[0.92750809 0.76963488 0.84682927]
The testing accuracy-score for <class 'sklearn.linear_model.logistic.LogisticRegression'> is: 
[0.92167577 0.73432836 0.81495327]

R2 scores for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> train set: 0.9534422981674097
R2 scores for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> test set: 0.9221598877980365
The training accuracy-score for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> is: 
[0.9752839  0.91408591 0.94125079]
The testing accuracy-score for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> is: 
[0.95712484 0.86075949 0.9009009 ]

R2 scores for <class 'sklearn.ensemble.bagging

In [26]:
for model in [logreg_class_pca, knn_class_pca, bagged_class_pca, random_forest_class_pca, 
              adaboost_class_pca] :
    accuracy_scorer(model, Z_train, Z_test, y_train, y_test)

R2 scores for <class 'sklearn.linear_model.logistic.LogisticRegression'> train set: 0.8705720630221
R2 scores for <class 'sklearn.linear_model.logistic.LogisticRegression'> test set: 0.8579988016776513
The training accuracy-score for <class 'sklearn.linear_model.logistic.LogisticRegression'> is: 
[0.92443411 0.65917843 0.76085106]
The testing accuracy-score for <class 'sklearn.linear_model.logistic.LogisticRegression'> is: 
[0.92110377 0.62611807 0.69902913]

R2 scores for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> train set: 0.9405731204398858
R2 scores for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> test set: 0.9185140802875974
The training accuracy-score for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> is: 
[0.9671062  0.85779685 0.87585682]
The testing accuracy-score for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> is: 
[0.95683453 0.7960199  0.82403433]

R2 scores for <class 'sklearn.ensemble.bagging.Ba

In [46]:
naivebayes_pca = MultinomialNB()
naivebayes_pca.fit(X_train, y_train)
accuracy_scorer(naivebayes_pca, X_train, X_test, y_train, y_test);

R2 scores for <class 'sklearn.naive_bayes.MultinomialNB'> train set: 0.7052996532937098
R2 scores for <class 'sklearn.naive_bayes.MultinomialNB'> test set: 0.6963534361851332
The training accuracy-score for <class 'sklearn.naive_bayes.MultinomialNB'> is: 
[0.83175237 0.34806071 0.63997478]
The testing accuracy-score for <class 'sklearn.naive_bayes.MultinomialNB'> is: 
[0.82530795 0.29540918 0.64424779]



In [47]:
y_pred_rf = random_forest_class_pca.predict(Z_train)
print(confusion_matrix(y_train, y_pred_rf))
print()
y_pred_rf = random_forest_class_pca.predict(Z_test)
print(confusion_matrix(y_test, y_pred_rf))

[[4479    2    0]
 [   6 1994    1]
 [   1    7 1586]]

[[777  12   2]
 [ 28 318   7]
 [  9  12 261]]


In [48]:
y_pred_bag = bagged_class_pca.predict(Z_train)
print(confusion_matrix(y_train, y_pred_bag))
print()
y_pred_bag = bagged_class_pca.predict(Z_test)
print(confusion_matrix(y_test, y_pred_bag))

[[4480    1    0]
 [  14 1986    1]
 [   2    6 1586]]

[[779   9   3]
 [ 39 307   7]
 [  3  15 264]]


### Testing 3 brackets of Income (0-20k, 20k-50k, 51k+)

In [49]:
print(y_allcities[y_allcities < 20000].shape)
print(y_allcities[(y_allcities >= 20000) & (y_allcities < 50000)].shape)
print(y_allcities[y_allcities >= 50000].shape)

(3779,)
(3254,)
(2469,)


In [50]:
## Creating column y_train_class3

y_train = y_allcities

y_train_class3 = pd.Series(['NaN' for i in range(len(y_train)) ])

## Creating y_train for classification

brackets ={'$0-$20k': 0,'$20-$50k':1, '$50+k':3}

for index, value in enumerate(y_train):
    if y_train.iloc[index] < 20000:
        y_train_class3.iloc[index] = brackets['$0-$20k']
    elif y_train.iloc[index] < 50000:
        y_train_class3.iloc[index] = brackets['$20-$50k']
    elif y_train.iloc[index] >= 50000:
        y_train_class3.iloc[index] = brackets['$50+k']
    else:
        y_train_class3.iloc[index] = np.nan

In [51]:
# Also with stratifying on income
X_train, X_test, y_train, y_test = train_test_split(X_allcities, y_train_class3, 
                                                    test_size=.15, random_state=42,
                                                   stratify=y_train_class3) 
X_train.shape

ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

## Applying PCA to Xs
pca = PCA(n_components=200)
pca = pca.fit(X_train_scaled)
Z_train = pca.transform(X_train_scaled)
Z_test =pca.transform(X_test_scaled)

In [52]:
logreg_class_pca = LogisticRegression()
logreg_class_pca.fit(Z_train, y_train)

knn_class_pca = KNeighborsClassifier()
knn_class_pca.fit(Z_train, y_train)

bagged_class_pca = BaggingClassifier()
bagged_class_pca.fit(Z_train, y_train)

random_forest_class_pca = RandomForestClassifier()
random_forest_class_pca.fit(Z_train, y_train)

adaboost_class_pca = AdaBoostClassifier()
adaboost_class_pca.fit(Z_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [53]:
for model in [logreg_class_pca, knn_class_pca, bagged_class_pca, random_forest_class_pca, 
              adaboost_class_pca] :
    accuracy_scorer(model, Z_train, Z_test, y_train, y_test)

R2 scores for <class 'sklearn.linear_model.logistic.LogisticRegression'> train set: 0.8784051510648836
R2 scores for <class 'sklearn.linear_model.logistic.LogisticRegression'> test set: 0.8744740532959326
The training accuracy-score for <class 'sklearn.linear_model.logistic.LogisticRegression'> is: 
[0.90005879 0.82575758 0.91052114]
The testing accuracy-score for <class 'sklearn.linear_model.logistic.LogisticRegression'> is: 
[0.90954357 0.81400438 0.89222374]

R2 scores for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> train set: 0.9593858345715701
R2 scores for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> test set: 0.9368863955119214
The training accuracy-score for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> is: 
[0.97158026 0.94518094 0.95967355]
The testing accuracy-score for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> is: 
[0.95154185 0.91411043 0.94451962]

R2 scores for <class 'sklearn.ensemble.bagging

In [36]:
for model in [logreg_class_pca, knn_class_pca, bagged_class_pca, random_forest_class_pca, 
              adaboost_class_pca] :
    accuracy_scorer(model, Z_train, Z_test, y_train, y_test)

R2 scores for <class 'sklearn.linear_model.logistic.LogisticRegression'> train set: 0.7758274294173628
R2 scores for <class 'sklearn.linear_model.logistic.LogisticRegression'> test set: 0.7495506291192331
The training accuracy-score for <class 'sklearn.linear_model.logistic.LogisticRegression'> is: [0.83234531 0.68824749 0.7428262 ]
The testing accuracy-score for <class 'sklearn.linear_model.logistic.LogisticRegression'> is: [0.81837161 0.64845173 0.68518519]

R2 scores for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> train set: 0.9212223749603469
R2 scores for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> test set: 0.89035350509287
The training accuracy-score for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> is: [0.9429083  0.8993943  0.88761358]
The testing accuracy-score for <class 'sklearn.neighbors.classification.KNeighborsClassifier'> is: [0.92359551 0.86218487 0.82065217]

R2 scores for <class 'sklearn.ensemble.bagging.Baggi

In [54]:
random_forest_class_pca = RandomForestClassifier()
random_forest_class_pca.fit(Z_train, y_train)

y_pred_rf = random_forest_class_pca.predict(Z_test)
confusion_matrix(y_test, y_pred_rf)

array([[557,  10,   0],
       [ 14, 456,  18],
       [  5,  19, 347]], dtype=int64)

In [87]:
joblib.dump(knn_class_pca, 'knn_pca_0-20k-50k+.joblib', compress=9)

['knn_pca_0-20k-50k+.joblib']

In [55]:
joblib.dump(random_forest_class_pca, 'rf_pca_0-20k-50k+_2ndcities.joblib', compress=9)

['rf_pca_0-20k-50k+_2ndcities.joblib']

In [56]:
joblib.dump(logreg_class_pca, 'log_pca_0-20k-50k+_2ndcities.joblib', compress=9)

['log_pca_0-20k-50k+_2ndcities.joblib']